# Testing multiple models with different levels of feature engineering

In [1]:
import pickle
import pathlib
import warnings
import numpy as np
import pandas as pd

In [2]:
DATA_DIR = pathlib.Path.cwd() / 'data'
print(DATA_DIR)

/Users/enriccogemha/Developer/HousePriceRegressor/data


In [3]:
clean_data_path = DATA_DIR / 'processed' / 'ames_model_data.pkl'

In [4]:
with open(clean_data_path, 'rb') as file:
    model_data = pickle.load(file)

In [5]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Columns: 165 entries, Lot.Frontage to Exterior_Other
dtypes: bool(119), float64(34), int64(12)
memory usage: 1.4 MB


In [6]:
# Create dictionary to store the results for each model

model_results = {}

## Train-test splitting

In [7]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [8]:
X, y

(      Lot.Frontage  Lot.Area  Lot.Shape  Land.Slope  Overall.Qual  \
 0            141.0   31770.0          1           0             5   
 1             80.0   11622.0          0           0             4   
 2             81.0   14267.0          1           0             5   
 3             93.0   11160.0          0           0             6   
 4             74.0   13830.0          1           0             4   
 ...            ...       ...        ...         ...           ...   
 2925          37.0    7937.0          1           0             5   
 2926          68.0    8885.0          1           1             4   
 2927          62.0   10441.0          0           0             4   
 2928          77.0   10010.0          0           1             4   
 2929          74.0    9627.0          0           1             6   
 
       Overall.Cond  Mas.Vnr.Area  Exter.Qual  Exter.Cond  BsmtFin.SF.1  ...  \
 0                4         112.0           2           2         639.0  ...  

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
RANDOM_SEED = 42  # Any number here, really.

In [11]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [12]:
X.shape, Xtrain.shape, Xtest.shape

((2877, 164), (2157, 164), (720, 164))

In [13]:
y.shape, ytrain.shape, ytest.shape

((2877,), (2157,), (720,))

## Primeiro teste: modelo linear com scaling e cross-validation

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


float64_columns = Xtrain.select_dtypes('float64').columns

col = ColumnTransformer(
    [
        ('scale', StandardScaler(), float64_columns),
    ],
    remainder='passthrough',
)

col.fit(Xtrain)

Xtrain_scaled = col.transform(Xtrain)
Xtest_scaled = col.transform(Xtest)


In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

lr = LinearRegression()

grid = {
    'fit_intercept': [True, False],
    'positive': [True, False],
}

grid_search = GridSearchCV(
    lr,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [16]:
grid_search.fit(Xtrain_scaled, ytrain)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Best parameters: {'fit_intercept': True, 'positive': True}
Best score: -0.0031667526802596456
Average error is 13.83%


In [17]:
model_results['LinearRegression'] = round(error_percent, 3)

Podemos perceber que existe um erro menor para o modelo com cross-validation e scaling, portanto iremos adotar isso para o restante dos experimentos.

## Segundo experimento: Modelo Lasso com cross-validation e scaling

O modelo Lasso, que significa "Least Absolute Shrinkage and Selection Operator" (Operador de Contração e Seleção Mínimas de Absoluto), é um método de regressão linear que inclui uma regularização L1. A regularização L1 envolve a adição de um termo de penalização à função objetivo da regressão, que incentiva a maioria dos coeficientes a serem exatamente iguais a zero. Isso torna o Lasso uma técnica eficaz para seleção de recursos e para lidar com problemas de regressão com alta dimensionalidade.

In [18]:
from sklearn.linear_model import Lasso

grid = {
    'alpha': np.logspace(-8, -3, 200),
}

lasso = Lasso(random_state=RANDOM_SEED)

grid_search = GridSearchCV(
    lasso,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [19]:
warnings.filterwarnings('ignore')
grid_search.fit(Xtrain_scaled, ytrain)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

Best parameters: {'alpha': 8.804883581643464e-05}
Best score: -0.003296308851327881


In [20]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is 14.13%


In [21]:
model_results['Lasso'] = round(error_percent, 3)

Percebe-se que Lasso tem menor erro que o linear, portanto iremos adotar o Lasso para o restante dos experimentos.

## Terceiro experimento: Lasso + fazer transformação de log para features com skewness (assimetria)

### Justificativa para realizar uma feature engineering a mais no modelo


Além do trabalho realizado pelo Professor [Fabio J. Ayres](https://github.com/FabioAyresInsper) no *dataset* Ames, realizamos a seguinte feature engineering:

- **Transformação de log para features com skewness (assimetria)**, já que ela é conhecida por fazer uma feature que tem distribuição assimétrica ter uma distribuição mais próxima da normal, o que é desejável para modelos de regressão linear. Optamos por realizá-la, pois no `02_analysis_and_preprocessing.ipynb` é possível observar um comportamente de assimetria no histogramas de algumas features. Como exemplo, vide o histograma da feature `Lot.Area`.

Além disso, a transformação de log atenua o impacto de outliers, uma vez que "espreme" os valores extremamente altos em direção aos valores médios. Isso pode ser útil para reduzir a influência de outliers nos resultados. Isso pode ser observado em alguns histogramas, como o da feature `Enclosed.Porch`, em que a assimetria se dá principalmente por conta de um *outlier*.

**DISCLAIMER: Vamos aplicar um threshold e realizar a transformação de log somente nas features com maior assimetria. Para isso, usaremos um threshold de 3. Não existe um indício claro que esse feature engineering trará resultado positivo, contudo certamente não trará nenhum impacto negativo.**

Vamos testar *skewness* para todas as colunas e adquire o index daquelas que possuem *skewness* > 3.

In [22]:
from scipy.stats import skew

skewness = Xtrain.select_dtypes('float64').apply(skew)

skewness

Lot.Frontage        1.374507
Lot.Area           13.935570
Mas.Vnr.Area        2.584621
BsmtFin.SF.1        1.013526
BsmtFin.SF.2        4.222772
Bsmt.Unf.SF         0.955611
Total.Bsmt.SF       0.729935
X1st.Flr.SF         1.263409
X2nd.Flr.SF         0.842831
Low.Qual.Fin.SF    11.186456
Gr.Liv.Area         1.096900
Bsmt.Full.Bath      0.614981
Bsmt.Half.Bath      3.940715
Full.Bath           0.159638
Half.Bath           0.686145
Bedroom.AbvGr       0.408400
Kitchen.AbvGr       4.018729
TotRms.AbvGrd       0.770448
Fireplaces          0.730059
Garage.Cars        -0.179943
Garage.Area         0.264373
Wood.Deck.SF        1.974402
Open.Porch.SF       2.676164
Enclosed.Porch      4.144894
X3Ssn.Porch        11.380044
Screen.Porch        3.984604
Pool.Area          18.402387
Misc.Val           23.239878
Mo.Sold             0.210787
Yr.Sold             0.156685
Garage.Age          0.639825
Remod.Age           0.432224
House.Age           0.570505
dtype: float64

In [23]:
skewness = skewness[abs(skewness) > 3]

skew_features = Xtrain[skewness.index]

skew_features.columns

Index(['Lot.Area', 'BsmtFin.SF.2', 'Low.Qual.Fin.SF', 'Bsmt.Half.Bath',
       'Kitchen.AbvGr', 'Enclosed.Porch', 'X3Ssn.Porch', 'Screen.Porch',
       'Pool.Area', 'Misc.Val'],
      dtype='object')

In [24]:
def log_tf(feature):
    return np.log1p(feature)

Xtrain_skew = Xtrain.copy()
Xtest_skew = Xtest.copy()

Xtrain_skew[skew_features.columns] = Xtrain_skew[skew_features.columns].apply(log_tf)
Xtest_skew[skew_features.columns] = Xtest_skew[skew_features.columns].apply(log_tf)


In [25]:
col.fit(Xtrain_skew)

Xtrain_scaled = col.transform(Xtrain_skew)
Xtest_scaled = col.transform(Xtest_skew)

In [26]:
warnings.filterwarnings('ignore')
grid_search.fit(Xtrain_scaled, ytrain)

In [ ]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

In [ ]:
model_results['Lasso+Skew'] = round(error_percent, 3)

Podemos perceber que a transformação de log para features com *skewness* > 3 trouxe um resultado positivo para o modelo. Portanto, iremos adotar isso para o restante dos experimentos.

Decidimos testar abaixo o Elastic Net, pois ele é uma combinação entre Lasso e Ridge, já que o Lasso é um modelo que já testamos e trouxe bons resultados. Portanto, acreditamos que o Elastic Net trará resultados melhores que somente o Lasso.

## Quarto experimento: Elastic Net

O Elastic Net é um modelo de regressão linear que combina duas técnicas de regularização: a regularização L1 (Lasso) e a regularização L2 (Ridge). É usado para lidar com problemas de regressão, semelhante à regressão linear, mas com a adição dessas penalizações, o que torna o modelo mais robusto e ajuda a evitar problemas de overfitting. O Elastic Net é útil quando se lida com conjuntos de dados que têm muitas características (alta dimensionalidade) e algumas delas são altamente correlacionadas.

In [ ]:
from sklearn.linear_model import ElasticNet

grid = {
    'alpha': np.logspace(-8, -3, 10),
    'l1_ratio': np.linspace(0.01, 1, 50),
}

elastic = ElasticNet(random_state=RANDOM_SEED)

grid_search = GridSearchCV(
    elastic,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [ ]:
warnings.filterwarnings('ignore')
grid_search.fit(Xtrain_scaled, ytrain) # This takes the Xtrain_scaled with the skewness transformation

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

In [ ]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

In [ ]:
model_results['Elastic Net'] = round(error_percent, 3)

## Comparação entre os modelos

In [ ]:
print('| Model | Average Error |\n| --- | --- |\n')
for model, error in model_results.items():
    print(f'| {model} | {error} |')

**Considerando todas os experimentos realizados, o Elastic Net foi o modelo com menor erro percentual.**

## Rodando o modelo Elastic Net no dataset de teste

In [ ]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

## Comparação: baseline vs. melhor modelo (Elastic Net)

In [ ]:
error_baseline = 15.11
print(f"O delta de erro entre eles é: {error_baseline - model_results['Elastic Net']}%")

## Conclusão

### Quais as consequências do desempenho do modelo final para a aplicação de negócios?

O modelo final, Elastic Net, tem um erro percentual de aproximadamente 15%. O impacto deste percentual não é irrelevante, dado que pode corresponder ao valor de comissão do negócio de venda do imóvel, portanto, para uma aplicação em nível de vida real, é importante que o modelo seja melhorado, ou seja usado com cautela máxima, com o cliente havendo sido informado sobre a margem de erro previamente (algo como uma "estimativa grosseira").

### Quais features são mais importantes na determinação do preço do imóvel?

In [ ]:
from sklearn.feature_selection import RFECV

rfecv = RFECV(
    grid_search.best_estimator_,
    cv=5,
    scoring='neg_mean_squared_error',
)

rfecv.fit(Xtrain_scaled, ytrain)


In [ ]:
print(f'Optimal number of features: {rfecv.n_features_}')

print(f'Selected features: {Xtrain.columns[rfecv.support_]}')

In [ ]:
print(f'Feature ranking (importance): {rfecv.ranking_}')